In [1]:
import os
import sys
import io
import pandas as pd
import re


cwd = os.getcwd()
current_dir=os.getcwd()
parrent_dir=os.path.abspath(os.path.join(cwd, os.pardir))
sys.path.insert(1, current_dir+'/01-scripts')


import src_pdftojson as pdftojson


In [2]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
azure_doc_endpoint = ""
azure_doc_key = ("")

os.environ["LLAMA_CLOUD_API_KEY"] =""

azure_llm_endpoint=""
azure_llm_api_key=""


In [3]:
company_name="SANOFI"

In [4]:
path_to_sample_documents = "00-data/pdf_inputs/"+company_name+"_2024_test.pdf"


In [ ]:
company_name="SANOFI"

In [ ]:
companies=["Stellantis_2023","LOREAL_2023","totalenergies_2023"]

In [ ]:
for company_name in companies:
    path_to_sample_documents = "00-data/pdf_inputs/"+company_name+".pdf"

    #PyPDF
    print("**********************************************")
    print(company_name+" : PyPDF")
    output_txt_file_name="00-data/text_outputs/"+company_name+"_pypdf.json"
    pdftojson.pdf_to_json_pypdf(path_to_sample_documents, output_txt_file_name)

    #PyMuPdf
    print("**********************************************")
    print(company_name+" : PyMuPDF")
    output_txt_file_name="00-data/text_outputs/"+company_name+"_pymupdf.json"
    pdftojson.extract_text_with_pymupdf_to_json(path_to_sample_documents, output_txt_file_name)

    #Azure
    print("**********************************************")
    print(company_name+" : Azure Intelligence")

    output_txt_file_name="00-data/text_outputs/"+company_name+"_azure.json"
    pdftojson.pdf_to_json_azure(path_to_sample_documents, output_txt_file_name, azure_doc_key, azure_doc_endpoint)

    #LLamaParse
    print("**********************************************")
    print(company_name+" : LLamaParse")
    
    output_txt_file_name="00-data/text_outputs/"+company_name+"_llamaparse.json"
    pdftojson.extract_text_from_pdf_llamaparse_json(
        pdf_path=path_to_sample_documents,
        json_output_path=output_txt_file_name,
        parse_mode = "parse_page_with_lvm",
        vendor_model = "openai-gpt4o",
        verbose=True
    )

    #Extract pages as images + LLM
    print("**********************************************")
    print(company_name+" : Extract pages as images + LLM")
    markdown_prompt = """Analyze the content and structure of this page image.
    Extract all text and format it using Markdown syntax.
    Key formatting instructions:
    - Use Markdown headings (#, ##, ###, etc.) for titles and section headers.
    - Use bullet points (-) or numbered lists (1., 2.) for lists.
    - Preserve paragraph breaks.
    - Use *italic* or _italic_ for italics and **bold** or __bold__ for bold text if clearly distinguishable.
    - Represent tables using Markdown table syntax if possible, otherwise extract the content linearly.
    - Each Markdown table should start with ### BEGIN TABLE and end with ### END TABLE
    - Return only the markdown with no explanation text. Do not include delimiters like \`\`\`markdown or \`\`\`html.

    Focus on capturing the semantic structure and content accurately in Markdown format."""

    LLMs_to_test=["gpt-4o","mistral-small-2503","Llama-3.2-90B-Vision-Instruct","Phi-4-multimodal-instruct"]

    for llm in LLMs_to_test:
        print("**********************************************")
        print(company_name+" : Extract pages as images + LLM ==> "+llm)
        output_txt_file_name="00-data/text_outputs/"+company_name+"_llm2_"+llm+".json"

        pdftojson.extract_text_from_pdf_azure_vision_json(
                pdf_path=path_to_sample_documents,
                json_output_path=output_txt_file_name,
                prompt=markdown_prompt,
                azure_endpoint=azure_llm_endpoint,
                azure_api_key="",
                azure_model_name=llm,
                verbose=True

            #"DeepSeek-V3"
            #Phi-4
            #"Llama-3.3-70B-Instruct"
            )

# Tests

## PyPDF

In [ ]:
output_txt_file_name="00-data/text_outputs/"+company_name+"_2024_test_pypdf.json"
pdftojson.pdf_to_json_pypdf(path_to_sample_documents, output_txt_file_name)

## PyMuPdf

In [5]:
output_txt_file_name="00-data/text_outputs/"+company_name+"_2024_test_pymupdf.json"
pdftojson.extract_text_with_pymupdf_to_json(path_to_sample_documents, output_txt_file_name)

✅ Text extracted and saved to JSON: 00-data/text_outputs/SANOFI_2024_test_pymupdf.json


## Azure Document Intelligence

In [ ]:
azure_endpoint = ""
azure_key = ("")

output_txt_file_name="00-data/text_outputs/"+company_name+"_2024_test_azure.json"
pdftojson.pdf_to_json_azure(path_to_sample_documents, output_txt_file_name, azure_key, azure_endpoint)


## Extract tables and graphs as images + LLM

In [5]:
os.environ["OPENAI_API_KEY"] =""

In [5]:
en_prompt = """Analyze the provided image and identify its content (text, table, chart, or other). Then rephrase (in the same language as the text in the image) the information in the most appropriate way:

- If the image mainly contains text, present it as well-structured paragraphs.
- If the image contains a table, format it in Markdown while preserving its structure and content, followed by a summary of the table's contents.
- If the image contains a chart, describe all the information from the chart in bullet points.
- If the image contains a combination of these elements, adapt the format accordingly.
- If the image contains a blurry element, a logo, or photos of people,
- Do not start with an introductory phrase like “Here is a rephrasing...”.
- Do not add any title before, within, or after the table or chart.

Ensure that the result is clear, well-organized, and faithful to the original content of the image."""

In [6]:
fr_prompt = """Analyse l’image fournie et identifie son contenu (texte, tableau, graphique ou autre). Reformule ensuite (dans la même langue que celle du texte présent dans l’image) les informations de la manière la plus appropriée :

- Si l’image contient principalement du texte, présente-le sous forme de paragraphes bien structurés.
- Si l’image contient un tableau, reformate-le en Markdown en préservant sa structure et son contenu, suivi d’un résumé du contenu du tableau.
- Si l’image contient un graphique, décris toutes les informations du graphique sous forme de points clés.
- Si l’image contient une combinaison de ces éléments, adapte le format en conséquence.
- Si l’image contient un élément flou, un logo ou des photos de personnes,
- Ne commence pas par une phrase d’introduction du type “Voici une reformulation...”.
- N’ajoute aucun titre avant, dans ou après le tableau ou le graphique.

Assure-toi que le résultat soit clair, bien organisé et fidèle au contenu original de l’image."""

In [1]:
output_txt_file_name="00-data/text_outputs/"+company_name+"_2024_test_llm1_Phi-4.json"

pdftojson.pdf_to_json_LLM_11(
    path_to_sample_documents=path_to_sample_documents,
    output_json_file_name=output_txt_file_name,
    prompt=en_prompt,
    use_azure=True,
    azure_endpoint="",
    azure_api_key="",
    azure_model_name="Phi-4"
    #"Mistral-small"
)



NameError: name 'company_name' is not defined

## Extract pages as images + LLM

In [6]:
markdown_prompt = """Analyze the content and structure of this page image.
Extract all text and format it using Markdown syntax.
Key formatting instructions:
- Use Markdown headings (#, ##, ###, etc.) for titles and section headers.
- Use bullet points (-) or numbered lists (1., 2.) for lists.
- Preserve paragraph breaks.
- Use *italic* or _italic_ for italics and **bold** or __bold__ for bold text if clearly distinguishable.
- Represent tables using Markdown table syntax if possible, otherwise extract the content linearly.
- Each Markdown table should start with ### BEGIN TABLE and end with ### END TABLE
- Return only the markdown with no explanation text. Do not include delimiters like \`\`\`markdown or \`\`\`html.

Focus on capturing the semantic structure and content accurately in Markdown format."""

In [10]:
markdown_prompt_fr = """Analyse le contenu et la structure de cette image de page.
Extrait tout le texte et formate-le en utilisant la syntaxe Markdown.
Instructions de formatage clés :
- Utilise les titres Markdown (#, ##, ###, etc.) pour les titres et les en-têtes de section.
- Utilise des listes à puces (-) ou des listes numérotées (1., 2.) pour les listes.
- Préserve les sauts de paragraphe.
- Utilise *italique* ou _italique_ pour l’italique et **gras** ou __gras__ pour le texte en gras si c’est clairement identifiable.
- Représente les tableaux en utilisant la syntaxe Markdown pour les tableaux si possible, sinon extrais le contenu de manière linéaire.
- Chaque tableau en Markdown doit commencer par ### BEGIN TABLE et se terminer par ### END TABLE
- Ne retourne que le Markdown sans texte explicatif. N’inclus pas de délimiteurs comme \`\`\`markdown ou \`\`\`html.

Concentre-toi sur la capture précise de la structure sémantique et du contenu en format Markdown."""

In [11]:
output_txt_file_name="00-data/text_outputs/"+company_name+"_2024_test_llm22.json"


In [ ]:

pdftojson.extract_text_from_pdf_gpt4o_json(
        pdf_path=path_to_sample_documents,
        json_output_path=output_txt_file_name,
        prompt=markdown_prompt,
        verbose=True
    )

In [ ]:
output_txt_file_name="00-data/text_outputs/"+company_name+"_2024_test_llm2_mistral-small-2503-2.txt"

pdftojson.extract_text_from_pdf_azure_vision_json(
        pdf_path=path_to_sample_documents,
        json_output_path=output_txt_file_name,
        prompt=markdown_prompt,
    
        azure_endpoint="",
        azure_api_key="",
    
        azure_model_name="mistral-small-2503-2",
        verbose=True
    #Phi-4
    #"Llama-3.3-70B-Instruct"
    )

## LLAMAPARSE